# htFuncLib
Welcome to htFuncLib!
The purpose of this notebook is to run the htFuncLib algorithm.
It assumes that you have already downloaded the PSSM, refined structure and resfiles from the [FuncLib](https://funclib.weizmann.ac.il/bin/steps) webserver.

htFuncLib consists these steps:
1. Running FuncLib (possibly on hte webserver).
2. Retrieving the output files.
3. Anslysis of the resfiles to determine the sizes of the various neighborhoods

## imports and initial setup

In [39]:
import os
import random
from copy import deepcopy
from glob import glob
from itertools import product
from pathlib import Path
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Bio.PDB.PDBParser import PDBParser
from tqdm.notebook import tqdm

In [67]:
random.seed(42)

# load basic parameters, this requires no changes
AAs = list("ACDEFGHIKLMNPQRSTVWY")

BB_ATOMS = ["N", "CA", "C", "O"]
DDGS = np.arange(0, 6.5, 0.5)
ONE_2_THREE = {
    "A": "ALA",
    "C": "CYS",
    "D": "ASP",
    "E": "GLU",
    "F": "PHE",
    "G": "GLY",
    "H": "HIS",
    "I": "ILE",
    "K": "LYS",
    "L": "LEU",
    "M": "MET",
    "N": "ASN",
    "P": "PRO",
    "Q": "GLN",
    "R": "ARG",
    "S": "SER",
    "T": "THR",
    "V": "VAL",
    "W": "TRP",
    "Y": "TYR",
}
THREE_2_ONE = {v: k for k, v in ONE_2_THREE.items()}

In [14]:
# bubble class:
class Bubble:
    def __init__(self, name, nodes: List[int] = [], extended: List[int] = []):
        self.name = name
        self.nodes = nodes
        self.extended = extended

    def __repr__(self) -> str:
        return f"{self.name} {self.nodes} {len(self.nodes)} {self.extended} {len(self.extended)}"

    def add_node(self, node):
        if isinstance(node, list):
            self.nodes += node
        else:
            self.nodes += [node]

    def add_extended(self, ext):
        if isinstance(ext, list):
            self.extended += ext
        else:
            self.extended += [ext]

    def add_extensions(self, extensions: List[int]) -> None:
        for ext in extensions:
            self.add_extended(ext)

    def remove_node(self, node):
        self.nodes.remove(node)

    def remove_extension(self, extension):
        self.extended.remove(extension)

    def remove_extensions(self, to_remove: List[int]) -> None:
        for to_ in to_remove:
            try:
                self.remove_extension(to_)
            except:
                raise ValueError("could not remove %i from %r" % (to_, self.extended))

    def get_all_nodes(self):
        return self.nodes + self.extended

In [15]:
# define some utility functions
def create_dir(name):
    if Path(name).exists():
        return
    else:
        os.mkdir(name)


def get_residue_by_num_chain(pdb, num_chain):
    for res in pdb.get_residues():
        if res.full_id[2] == num_chain[-1] and str(res._id[1]) == num_chain[:-1]:
            return res


def calc_min_residue_distance(pdb, num_chain1, num_chain2):
    res1 = get_residue_by_num_chain(pdb, num_chain1)
    res2 = get_residue_by_num_chain(pdb, num_chain2)
    min_dist = np.inf
    for a1 in res1.get_atoms():
        for a2 in res2.get_atoms():
            dist = np.linalg.norm(a1.get_coord() - a2.get_coord())
            min_dist = dist if dist < min_dist else min_dist
    return min_dist


def count_residues_closer_than_distance(
    pdb, num_chain1, num_chain2, distance_threshold, avoid_bb
):
    res1 = get_residue_by_num_chain(pdb, num_chain1)
    res2 = get_residue_by_num_chain(pdb, num_chain2)
    count = 0
    for a1 in res1.get_atoms():
        if avoid_bb and a1.name in BB_ATOMS:
            continue
        for a2 in res2.get_atoms():
            if avoid_bb and a2.name in BB_ATOMS:
                continue
            dist = np.linalg.norm(a1.get_coord() - a2.get_coord())
            if dist <= distance_threshold:
                count += 1
    return count


def parse_resfile(f):
    df = pd.read_csv(f, sep="\s+", skiprows=2, names=["pos", "chain", "PIKAA", "AAs"])
    df["len_AAs"] = df.AAs.map(len)
    df["num_chain"] = df["pos"].astype(str) + df["chain"]
    return df

## setting up the initial paramteres. this is where you use the files you got from FuncLib

In [45]:
# fields for user input

# path in which to create files and folders
W_PATH = Path("./").absolute()
# path to Rosetta scripts executable
R_PATHS = "PATH_TO_YOUR_ROSETTA_SCRIPTS_EXECUTABLE"
# path to the refined PDB, retrieved from FuncLib
PDB_FILE = os.path.abspath(f"{W_PATH}/initial_data/refined.pdb")
pdb = PDBParser().get_structure("", PDB_FILE)
# the PDB chain to work on
CHAIN = "A"
# positions to work on
ALL_POSS = [
    42,
    44,
    46,
    61,
    64,
    66,
    68,
    69,
    110,
    112,
    145,
    150,
    163,
    165,
    167,
    181,
    201,
    220,
    224,
    14,
    16,
    18,
    72,
    108,
    119,
    123,
    185,
]
ALL_POSS_CHAIN = [f"{a}{CHAIN}" for a in ALL_POSS]
# path to the PSSM, retrieved from FuncLib
PSSM = f"{W_PATH}/initial_data/pssm"

RESFILES = {
    float(str(a)[-3:]): W_PATH / a
    for a in (W_PATH / "initial_data").glob("designable_aa_resfile.*.*")
}

LIGS = ["GYS"]
flags_file = W_PATH / "initial_data/flags"


def parse_flags(flags_file):
    flgs_to_ignore = ["database"]
    for l in open(flags_file):
        print(l.rstrip())
    return


flags = parse_flags(flags_file)

-database /home/labs/fleishman/rosaliel/Rosetta/main/database
-pdb_gz
-overwrite
-use_input_sc
-extrachi_cutoff 5
-ignore_unrecognized_res
-chemical:exclude_patches LowerDNA UpperDNA Cterm_amidation SpecialRotamer VirtualBB ShoveBB VirtualDNAPhosphate VirtualNTerm CTermConnect sc_orbitals pro_hydroxylated_case1 pro_hydroxylated_case2 ser_phosphorylated thr_phosphorylated tyr_phosphorylated tyr_sulfated lys_dimethylated lys_monomethylated lys_trimethylated lys_acetylated glu_carboxylated cys_acetylated tyr_diiodinated N_acetylated C_methylamidated MethylatedProteinCterm
-linmem_ig 10
-ignore_zero_occupancy false
-mute all
-no_nstruct_label true
-in:file:native initial_data/refined.pdb
-extra_res_fa initial_data/LG.params
-s initial_data/refined.pdb
-parser:protocol initial_data/mutate.xml
-pdb_gz
-parser:script_vars res_to_fix=94A,96A,121A,148A,203A,205A,222A,1X
-parser:script_vars fix_prefix=<
-parser:script_vars fix_TO=,fix_res
-parser:script_vars cst_full_path=initial_data/coord.cst


## create the bubbles (neighborhoods)

In [17]:
# create bubbles
bubbles = dict((p, Bubble(name=p, nodes=[p], extended=[])) for p in ALL_POSS_CHAIN)

# extend each bubble to all positions that have at least 2 non-backbone atoms within 6A it.
# you can change the thresholds if you wish
for bub in bubbles.values():
    for p in ALL_POSS_CHAIN:
        if p not in bub.get_all_nodes():
            if count_residues_closer_than_distance(
                pdb, bub.nodes[0], p, 6, avoid_bb=True
            ):
                bub.add_extended(p)

# some bubble contain too many positions that are not trully interacting with the bubble center
# you can list those here, and they will be removed.
bubbles["42A"].remove_extensions(["66A", "69A"])
bubbles["46A"].remove_extensions(["61A", "18A"])
bubbles["64A"].remove_extensions(["66A", "68A"])
bubbles["66A"].remove_extensions(["42A", "61A", "64A", "68A", "69A", "224A"])
bubbles["68A"].remove_extensions(["16A", "66A", "69A", "72A", "220A"])
bubbles["69A"].remove_extensions(["42A", "66A", "112A", "68A", "224A"])
bubbles["72A"].remove_extensions(["68A", "69A"])

bubbles["112A"].remove_extensions(["69A", "123A"])
bubbles["165A"].remove_extensions(["66A", "145A"])
bubbles["224A"].remove_extensions(["66A", "150A"])

# use the printed selection statements to selct and view the various bubbles in PyMOL.
# use PyMOL to see which positions are in which bubble, so you can safely remove them.
for bub in bubbles.values():
    print(f"select bub_{bub.name}, resi {'+'.join([str(a[:-1]) for a in bub.nodes])}")
    print(
        f"select ext_{bub.name}, resi {'+'.join([str(a[:-1]) for a in bub.extended])}"
    )

select bub_42A, resi 42
select ext_42A, resi 44+68+220+224+14+72+119
select bub_44A, resi 44
select ext_44A, resi 42+46+64+66+68+220+14+16+119
select bub_46A, resi 46
select ext_46A, resi 44+64+68+220+16+123
select bub_61A, resi 61
select ext_61A, resi 46+64+66+145+167+220
select bub_64A, resi 64
select ext_64A, resi 44+46+61+220+16+18+123
select bub_66A, resi 66
select ext_66A, resi 44+145+150+165+167+220
select bub_68A, resi 68
select ext_68A, resi 42+44+46+64+112+224+14+119
select bub_69A, resi 69
select ext_69A, resi 150+163+165+201+72+185
select bub_110A, resi 110
select ext_110A, resi 112+18+108+123
select bub_112A, resi 112
select ext_112A, resi 68+110+14+119+185
select bub_145A, resi 145
select ext_145A, resi 61+66+165+167+181+220
select bub_150A, resi 150
select ext_150A, resi 66+69+163+165+167+181+201+224
select bub_163A, resi 163
select ext_163A, resi 69+150+165+181+201+185
select bub_165A, resi 165
select ext_165A, resi 69+150+163+167+181
select bub_167A, resi 167
select ex

In [18]:
# create bubble resfiles
# each bubble will have it's own resfile for each ∆∆G threhsold
BUB_RESFILES = {}
bubb_resfiles_path = W_PATH / "bubb_resfiles"
create_dir(bubb_resfiles_path)
wt_pos_aa = dict(
    (p, THREE_2_ONE[get_residue_by_num_chain(pdb, p).resname]) for p in ALL_POSS_CHAIN
)
for fs, ori_rf_file in RESFILES.items():
    ori_rf_main = parse_resfile(ori_rf_file)
    BUB_RESFILES[fs] = {}
    for bub in bubbles.values():
        ori_rf = deepcopy(ori_rf_main)

        resfile = {
            p: aas for p, aas in zip(ori_rf.num_chain, ori_rf.AAs) if p in bub.extended
        }
        for pos, aa in wt_pos_aa.items():
            if pos not in resfile.keys():
                resfile[pos] = aa
        if all([len(resfile[p]) == 1 for p in ori_rf.num_chain]):
            continue
        resfile = dict(sorted(resfile.items(), key=lambda x: int(x[0][:-1])))

        rf_file = f"{str(bubb_resfiles_path)}/bub{bub.name}_{fs}.resfile"
        with open(rf_file, "w+") as fout:
            fout.write("nataa\nstart\n")
            for k, v in resfile.items():
                fout.write(f"{k[:-1]: <8}{CHAIN}       PIKAA   {v}\n")
        BUB_RESFILES[fs][bub.name] = rf_file

In [19]:
# running this block will print how many jobs will be required to cover each bubble for every ∆∆G
# threshold.
count_by_ddg = dict((ddg, 0) for ddg in sorted(BUB_RESFILES.keys()))
for ddg in count_by_ddg.keys():
    jobs_per_ddg = 0
    all_perms[ddg] = {}
    for bub in bubbles.values():
        if bub.name not in BUB_RESFILES[ddg].keys():
            continue
        len_perms = np.product(
            parse_resfile(BUB_RESFILES[ddg][bub.name]).len_AAs.tolist()
        )
        count_by_ddg[ddg] += len_perms
        jobs = len_perms if len_perms * 0.15 < 1000 else round(len_perms * 0.15)
        jobs_per_ddg += jobs
        print(
            f"bubble {bub.name} ext {len(bub.extended)} dG {ddg:.1f} perms: {len_perms} jobs: {jobs}"
        )
    print(f"∆∆G {ddg:.1f} {count_by_ddg[ddg]} #jobs {jobs_per_ddg}")
    print("*****************************************************************")
    if count_by_ddg[ddg] > 10000000:
        break

bubble 42A ext 7 dG 0.0 perms: 8 jobs: 8
bubble 44A ext 9 dG 0.0 perms: 4 jobs: 4
bubble 46A ext 6 dG 0.0 perms: 8 jobs: 8
bubble 61A ext 6 dG 0.0 perms: 2 jobs: 2
bubble 64A ext 7 dG 0.0 perms: 8 jobs: 8
bubble 66A ext 6 dG 0.0 perms: 4 jobs: 4
bubble 69A ext 6 dG 0.0 perms: 8 jobs: 8
bubble 110A ext 4 dG 0.0 perms: 20 jobs: 20
bubble 112A ext 5 dG 0.0 perms: 2 jobs: 2
bubble 145A ext 6 dG 0.0 perms: 2 jobs: 2
bubble 150A ext 8 dG 0.0 perms: 4 jobs: 4
bubble 163A ext 6 dG 0.0 perms: 8 jobs: 8
bubble 165A ext 5 dG 0.0 perms: 8 jobs: 8
bubble 167A ext 6 dG 0.0 perms: 8 jobs: 8
bubble 181A ext 5 dG 0.0 perms: 4 jobs: 4
bubble 201A ext 5 dG 0.0 perms: 16 jobs: 16
bubble 220A ext 8 dG 0.0 perms: 4 jobs: 4
bubble 224A ext 5 dG 0.0 perms: 16 jobs: 16
bubble 14A ext 6 dG 0.0 perms: 4 jobs: 4
bubble 16A ext 8 dG 0.0 perms: 8 jobs: 8
bubble 18A ext 6 dG 0.0 perms: 20 jobs: 20
bubble 108A ext 3 dG 0.0 perms: 4 jobs: 4
bubble 119A ext 6 dG 0.0 perms: 4 jobs: 4
bubble 123A ext 7 dG 0.0 perms: 20 j

## now it's time to choose a ∆∆G threshold
we recommend to sticking to <10^6 jobs, so you don't wait too long.
the above blocks calcualte the number of jobs required for each ∆∆G threshold.
use their output to choose.
if you are unsatisfied with the sequence space, try pruning more positions from the bubbles that require the most jobs to model.

In [25]:
CHOSEN_DDG = 5.5
chosen_rf_df = parse_resfile(RESFILES[CHOSEN_DDG])
chosen_rf_df

,pos,chain,PIKAA,AAs,len_AAs,num_chain
0,14,A,PIKAA,IV,2,14A
1,16,A,PIKAA,VILMT,5,16A
2,18,A,PIKAA,LFI,3,18A
3,42,A,PIKAA,LIMV,4,42A
4,44,A,PIKAA,LIMV,4,44A
5,46,A,PIKAA,FY,2,46A
6,61,A,PIKAA,VALMQT,6,61A
7,64,A,PIKAA,L,1,64A
8,66,A,PIKAA,YAFHLMNQSW,10,66A
9,68,A,PIKAA,VACILMQST,9,68A


## prepare and run the design jobs

In [70]:
DESIGN_PATH = "%s/bubb_design" % W_PATH
create_dir(DESIGN_PATH)

total = 0
for bub in tqdm(bubbles.values()):
    bub_nodes = sorted(bub.get_all_nodes(), key=lambda x: int(x[:-1]))

    combs = [
        list(chosen_rf_df.loc[chosen_rf_df["num_chain"] == n, "AAs"].values[0])
        for n in bub_nodes
    ]
    perms = list(product(*combs))
    if len(perms) > 1000:
        selected_combs = random.choices(perms, k=int(0.15*len(perms)))
    else:
        selected_combs = perms
    print(f'for bubble {bub.name}, creating {len(selected_combs)} jobs')
    total += len(selected_combs)
    create_dir(f'{DESIGN_PATH}/{bub.name}')
    create_dir(f'{DESIGN_PATH}/{bub.name}/scores')
    
    with open(f'{DESIGN_PATH}/{bub.name}/selected_combinations.txt', "w+") as fout:
        job = f"{R_PATHSPATHS} "
        job += f'@{W_PATH}/initial_data/flags ' 
        job += f'-out:prefix wt_ '
        job += f'-out:file:score_only {DESIGN_PATH}/{bub.name}/scores/wt.sc '

        job += f'-parser:script_vars '
        job += f'all_ress={",".join(f"{p}A" for p in sorted(ALL_POSS) if p != 66)} '
        job += "\n"
        
        fout.write(job)
        for perm in selected_combs:
            job = "rosetta_scripts "
            job += f'@{W_PATH}/initial_data/flags ' 
            job += f'-out:prefix {"_".join(p+aa for p, aa in zip(bub_nodes, perm))}_ '
            job += f'-out:file:score_only {DESIGN_PATH}/{bub.name}/scores/{"_".join(p+aa for p, aa in zip(bub_nodes, perm))}.sc '
            
            job += f'-parser:script_vars '
            job += f'all_ress={",".join(f"{p}A" for p in sorted(ALL_POSS) if p != 66)} '
            for i, (pos, aa) in enumerate(zip(bub_nodes, perm)):
                job += f'new_res{i+1}={ONE_2_THREE[aa]} target{i+1}={pos} '
            job += "\n"
            fout.write(job)
print(f'\nCreated a total of {total} jobs.')

  0%|          | 0/27 [00:00<?, ?it/s]

for bubble 42A, creating 46656 jobs
for bubble 44A, creating 86400 jobs
for bubble 46A, creating 1296 jobs
for bubble 61A, creating 4320 jobs
for bubble 64A, creating 2592 jobs
for bubble 66A, creating 21600 jobs
for bubble 68A, creating 12960 jobs
for bubble 69A, creating 12150 jobs
for bubble 110A, creating 1053 jobs
for bubble 112A, creating 2025 jobs
for bubble 145A, creating 51840 jobs
for bubble 150A, creating 648000 jobs
for bubble 163A, creating 10800 jobs
for bubble 165A, creating 2700 jobs
for bubble 167A, creating 64800 jobs
for bubble 181A, creating 2160 jobs
for bubble 201A, creating 4860 jobs
for bubble 220A, creating 124416 jobs
for bubble 224A, creating 17496 jobs
for bubble 14A, creating 5400 jobs
for bubble 16A, creating 9720 jobs
for bubble 18A, creating 2106 jobs
for bubble 72A, creating 864 jobs
for bubble 108A, creating 210 jobs
for bubble 119A, creating 5400 jobs
for bubble 123A, creating 10530 jobs
for bubble 185A, creating 750 jobs
Created a total of 1153104 jo

### This is where you run all jobs and aggregate the resulting scores.